## Введение в машинное обучение

## НИУ ВШЭ

### Домашнее задание №3

### О задании

В этом домашнем задании вы реализуете решающее дерево и попрактикуетесь в решении задач классификации.

### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
\text{points} \times 10 / 14,
$$

где points — количество баллов за обязательную часть, которое вы набрали.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

Импортируйте все нужные вам функции ниже:

In [146]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from typing import Iterable, List, Tuple, Union
from collections import Counter
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
plt.rcParams["figure.figsize"] = (11, 6.5)

# Решающее дерево своими руками (8 баллов + бонус)

В этой части для тестирования будем использовать датасет breast cancer. По предоставленной информации о ядрах клеток нужно предсказать присутствуют ли на изображении раковые клетки (класс 0) или нет (класс 1).

In [8]:
breast_cancer = load_breast_cancer()
X = pd.DataFrame(data=breast_cancer["data"], columns=breast_cancer["feature_names"])

# добавим искуственный категориальный признак
X['mean area cat'] = pd.qcut(X['mean area'], 5, labels=['smallest','small','medium','big', 'largest']).astype('object') 

X["target"] = breast_cancer["target"]
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)
X_train.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,mean area cat,target
287,12.89,13.12,81.89,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,0.05581,...,87.40,577.0,0.09616,0.1147,0.1186,0.05366,0.2309,0.06915,medium,1
512,13.40,20.52,88.64,556.7,0.11060,0.14690,0.14450,0.08172,0.2116,0.07325,...,113.30,844.4,0.15740,0.3856,0.5106,0.20510,0.3585,0.11090,medium,0
402,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,0.05899,...,96.31,621.9,0.09329,0.2318,0.1604,0.06608,0.3207,0.07247,medium,1
446,17.75,28.03,117.30,981.6,0.09997,0.13140,0.16980,0.08293,0.1713,0.05916,...,145.40,1437.0,0.14010,0.3762,0.6399,0.19700,0.2972,0.09075,largest,0
210,20.58,22.14,134.70,1290.0,0.09090,0.13480,0.16400,0.09561,0.1765,0.05024,...,158.30,1656.0,0.11780,0.2920,0.3861,0.19200,0.2909,0.05865,largest,0


### 1. Оцениванием качество разбиения (1 балл)

$R_m$ - множество объектов в разбиваемой вершине, $j$ - номер признака, по которому происходит разбиение, $t$ - порог разбиения.

Критерий ошибки:

$$
Q(R_m, j, t) = \frac{|R_\ell|}{|R_m|}H(R_\ell) + \frac{|R_r|}{|R_m|}H(R_r) \to \min_{j, t}
$$

$R_\ell$ - множество объектов в левом поддереве, $R_r$ - множество объектов в правом поддереве.

$H(R)$ - критерий информативности, с помощью которого можно оценить качество распределения целевой переменной среди объектов множества $R$.

Используйте функции для подсчета значения критерия ошибки, а также для разбиения вершины из семинара:

In [3]:
def split_node(R_m: np.ndarray, feature: str, t: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Split a fixed set of objects R_m with given feature name and threshold t
    """
    mask = R_m[feature] <= t
    R_l = R_m.loc[mask]
    R_r = R_m.loc[~mask]
    return R_l, R_r


def q_error(R_m: np.ndarray, feature: str, t: float) -> float:
    """
    Compute error criterion for the given objects R_m, feature name and threshold t
    """
    R_l, R_r = split_node(R_m, feature, t)
    return  len(R_l) / len(R_m) * H(R_l['target']) + len(R_r) / len(R_m) * H(R_r['target'])

__(0.5 балла)__ Реализуйте функцию для вычисления критерия информативности. На семинаре мы рассматривали решающее дерево для регрессии и в качестве критерия качества разбиения использовали дисперсию целевой переменной. Для классификации лучше использовать другие критерии, например энтропию:

$$H(R) = -p_0\log_{2}{p_0} -p_1\log_{2}{p_1},$$ где $p_1$, $p_0$ — доля объектов среди $R$, которые относятся к классу 1 и 0 соответственно.

In [4]:
def H(y: np.ndarray) -> float:
    """
    Compute entropy for vector y with classes of objects R
        
    """
    probabilities = # your code here

    return entropy

# Проверяем на простых примерах
assert np.isclose(H([0,0,0,0,0,1]), 0.650022)
assert np.isclose(H([0,0,0,0,0,0]), 0)

__(0.5 балла)__ Выберите признак, который как вам кажется может быть полезен для предсказания и порог для него. Сравните значение критерия информативности для объектов выборки до разбиения и взвешенной суммы критериев информативности для объектов после разбиения ($Q(R_m,j,t)$). Какой можно сделать вывод?

In [ ]:
# your code here

### 2. Ищем наилучшее разбиение (2 балла)

Теперь нужно найти наилучшее разбиение множества объектов $R_m$ в данной вершине, то есть такой порог $t$ для некоторого признака, где значение критерия ошибки $Q(R_m, j, t)$ минимально.

__(1 балл)__ Модифицируйте функцию *get_optimal_split* из семинара так, чтобы:
- Не было случаев, когда в одно из поддеревьев попадает 0 объектов 
- В качестве порога использовалось среднее двух различных соседних (при сортировке) значений признака 
- При одинаковых значениях критерия ошибки выбирался минимальный сплит 

__(Бонусные 0.5 балла)__ Перепешите функцию так, чтобы не использовались циклы.

In [143]:
def get_optimal_split(R_m: np.array, feature: str) -> Tuple[float, List[float]]:
    """
    Find best split of objects R_m by feature and return minimal q_error (opt_q_error), best threshold (opt_threshold) and array of error criterions (Q_array)
        
    """
    Q_array = []
    feature_values = np.unique(R_m[feature])
    
    for t in feature_values:
        Q_array.append(q_error(R_m, feature, t))
        
    Q_array = np.nan_to_num(Q_array, nan=float("+inf"))
    
    minimum_id = np.argmin(Q_array)
    opt_threshold = feature_values[minimum_id]
    opt_q_error = Q_array[minimum_id]
    
    return opt_threshold, opt_q_error, Q_array

__(0.25 балла)__ Постройте график зависимости критерия ошибки от выбранного порога для того же признака что и в 1ом задании и отметьте точку где достигается минимум.

In [ ]:
# your code here

__(0.75 балла)__ Найдите признак с минимальным значением критерия ошибки. Постройте на одном графике распределения значений этого признака для нулевого и первого класса (можно использовть seaborn) и добавьте прямую указывающую местоположение порога. 

In [ ]:
# your code here

### 3. Строим дерево (5 баллов)

Теперь можно реализовать алгоритм целиком. Начинаем строить дерево с корня. В корне дерева находится вся обучающая выборка. Затем используем жадный алгоритм:

0. Проверяем критерий остановки - все элементы в вершине относятся к одному классу, ни по одному признаку нельзя разбить выборку, достигнута максимальная глубина дерева и пр.

1. Cреди всех признаков выбираем признак с минимальным значением критерия ошибки.

2. Разбиваем выборку на две подвыборки по наилучшему порогу для этого признака и из этих подвыборок получаем две новые дочерние вершины. 

3. Для каждой из них рекурсивно потворяем аналогичные действия.

__(3 балла)__ Заполните пропущенные строчки в функции __fit_node_ и реализуйте функцию __predict_node_.

In [167]:
class DecisionTree:
    def __init__(
        self, 
        feature_types: Union[List[str], np.ndarray], 
        max_depth: int = None, 
        min_samples_split: int = None, 
        min_samples_leaf: int = None
    ) -> None:
        
        if np.any(list(map(lambda x: x not in ('int64', 'float64', 'object'), feature_types))):
            raise ValueError("There is unknown feature type")

        # В этой переменной будем хранить узлы решающего дерева. Каждая вершина хранит в себе идентификатор того,
        # является ли она листовой ("terminal" или "nonterminal"). Листовые вершины хранят значение класса для предсказания, 
        # нелистовые - правого и левого детей (поддеревья для продолжения процедуры предсказания)
        self._tree = {"depth":0}
        
        # типы признаков (категориальные или числовые)
        self._feature_types = feature_types
        
        # гиперпараметры дерева
        self._max_depth = max_depth
        self._min_samples_split = min_samples_split
        self._min_samples_leaf = min_samples_leaf

    def _fit_node(
        self, 
        sub: pd.DataFrame, # подмножество объектов для данной вершины
        node: dict        # словарь для хранения информации о вершине
    ) -> None:
        
        # критерий остановки - проверяем что не все классы объектов в данной вершине одинаковы
        if np.all(sub['target'] == sub['target'].iloc[0]):
            node["type"] = "terminal"
            node["class"] = sub['target'].iloc[0]
            return
        
        
        # ищем лучший признак для разбиения
        feature_best, threshold_best, q_best = None, None, None
        for feature in sub.columns[:-1]:
            feature_type = self._feature_types[feature]
              
            # ищем оптимальный порог для текущего признака
            threshold, q, q_array = get_optimal_split(sub, feature)
            
            
            # your code here

            
        # выбираем класс для листовой вершины
        if feature_best is None or node["depth"] == self._max_depth:
            node["type"] = "terminal"
            node["class"] = Counter(sub['target']).most_common(1)[0][0]
            return
        
        # записываем полученное разбиение в атрибуты класса
        node["type"] = "nonterminal"
        node["feature_split"] = feature_best
        node["threshold"] = threshold_best
        sub_l, sub_r = split_node(sub, feature_best, threshold_best)
    
        # запускаем рекурсию
        node["left_child"], node["right_child"] = {"depth": node["depth"]+1}, {"depth": node["depth"]+1}
        self._fit_node(sub_l, node["left_child"])
        self._fit_node(sub_r, node["right_child"])

    def _predict_node(self, x: pd.Series, node: dict) -> int:
        """
        Предсказание начинается с корневой вершины дерева и рекурсивно идёт в левое или правое поддерево в зависимости от значения
        предиката на объекте. Листовая вершина возвращает предсказание.
        :param x: pd.Series, элемент выборки
        :param node: dict, вершина дерева
        """
        # your code here
        

    def fit(self, X: pd.DataFrame, y: np.ndarray) -> None:
        X['target'] = y
        self._fit_node(X, self._tree)

    def predict(self, X: pd.DataFrame) -> pd.Series:
        predicted = []
        for ind, x in X.iterrows():
            predicted.append(self._predict_node(x, self._tree))
            
        return np.array(predicted)

__(1 балл)__ Обучите решающее дерево на обучающей части датасета (исключив колонку "mean area cat") и сравните accuracy полученную на обучающей и тестовой части. Совпадают ли топовые признаки с минимальным значением ошибки из предпредыдущего задания с признаками по которым произошли разбиения в дереве?

__(1 балл)__ Как будет происходить разбиение в вершине дерева по категориальному признаку? Является ли оно эффективным? Исправьте одну из функций выше так, чтобы дерево не выдавало ошибку при обучении на датасете с категориальными признаками. 

# Практическая часть (6 баллов)

__(2 балла)__ В этом задании нужно для того же датасета обучить несколько алгоритмов с помощью кросс-валидации и сравнить их качество по ROC AUC, accuracy и f1-score. Не забудьте удалить дополнительные колонки, которые были добавлены ранее.

1. Обучите и нарисуйте решающее дерево c глубиной 3. Настройте font_size или общий размер графика чтобы названия признаков были читабельны. Сравните его с деревом, которое вы написали самостоятельно. Для решающего дерева подберите оптимальный max_depth и min_samples_split по выбранной метрике.

2. Обучите логистическую регрессию c L2 регуляризацией и подберите для нее наилучший параметр.

3. Обучите SVM и выберите наиболее подходящее ядро и параметр регуляриации.


Выберите метрику, по которой вы будете выбирать наилучшие параметры. Почему для этого датасета стоит сравнивать предсказания не только по значению accuracy? Что важнее для этой задачи - оптимизировать precision или recall? 

Теперь загрузим еще один [датасет](https://www.kaggle.com/datasets/rashmiranu/banking-dataset-classification). По данным о клиентам банка нужно предсказать будет ли клиент брать кредит на длительный срок или нет. И если да, то сотрудники банка позвонят и предложат ему кредит. 

In [143]:
df = pd.read_csv('new_train.csv')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,999,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,999,0,nonexistent,no


__(0.5 балла)__ Изучите и подготовьте данные - проверьте типы колонок, соотношение классов, наличие пропусков (пропуски для категориальных переменных указаны как 'unknown'), повторяющихся объектов, проверьте частоты значений признаков и их смысл - возможно какие-то признаки можно удалить. 

__(0.5 балла)__ Постройте графики с распределениями по каждому из категориальных признаков. Статистики должны быть выведены для обоих классов и либо расположены на одном графике, либо находиться на соседних графиках, чтобы можно было сравнить их между собой. Вам поможет plt.subplot(s) если вы используете matplotlib или продвинутые функции из seaborn, позволяющие автоматически строить сразу несколько графиков. Убедитесь что на графиках подписаны оси, все надписи читабельны и пр. Проанализируйте полученные результаты.

__(0.5 балла)__ Проведите анализ числовых признаков - постройте попарные графики для признаков и график с корреляцией Пирсона.

__(0.5 балл)__ Преобразуйте категориальные признаки. Подумайте какие признаки лучше закодировать с помощью one-hot encoding, а какие с помощью label-encoding.

__(1 балл)__ В этом датасете отношение между положительными примерами и отрицательными практически 1:8. Поэтому хорошей идеей будет сбалансировать датасет (oversampling - добавить элементы менее популярного класса на основе имеющихся или undersampling - наоборот убрать элементы более популярного класса. см. [imblearn](https://imbalanced-learn.org/stable/over_sampling.html#a-practical-guide)) или пропорционально изменить веса классов в самих моделях (параметр class_weight), а также использовать чувствительные к таким случаям метрики. 

Разбейте датасет на train и test, используйте параметр stratify, чтобы соотношение классов не изменилось после разбиения датасета на две части. Аналогично в дальнейшем при использовании кросс-валидации используйте версию функции сохраняющую соотношение классов (StratifiedKFold). 

Подберите параметры логистической регрессии и решающего дерева с помощью кросс-валидации по AUC PR (площадь под Precision-Recall кривой). Для тестовой части датасета выведите получившиеся значения AUC PR, f1-score и выведите матрицы для истинных значений и предсказаний (confusion matrix). Их можно красиво вывести с помощью seaborn.heatmap(). На какую метрику стоит больше обращать внимание - на precision или recall?

__(0.5 балла)__ Рассмотрим задачу с точки зрения прибыли для банка. Маркетинговая компания требует значительных финансовых затрат, и ее эффективность напрямую зависит от качества нашей модели. Поэтому в качестве дополнительной метрики качества разумно использовать общую прибыль банка в той или иной форме. Мы будем рассматривать очень простую модель. Пусть каждый клиент после возврата всех процентов по кредиту (и с учетом всех расходов на обслуживание) приносит банку в среднем 10000 у.е., затраты на привлечение одного клиента составляют 100 у.е. Тогда сколько составит прибыль банка (доходы - расходы) если работники банка свяжутся со всеми клиентами, которых предсказала наша лучшая модель как подходящих на тестовой части и они все согласятся открыть кредит? Сколько составят расходы на маркетинг? В данной модели мы не учитываем что кредит может быть не возвращен в срок.

__(0.5 балла)__ Снова используйте кросс-валидацию с пятью подвыборками. Обучите логистическую регрессию и выведите пять значений прибыли, а также подсчитайте среднее значение. Постройте графики зависимости среднего значения прибыли и AUC PR от параметра регуляризации.